In [11]:
import pandas as pd
import io
import requests
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [12]:
url = "https://static.data.gouv.fr/resources/demandes-de-valeurs-foncieres/20230405-154703/valeursfoncieres-2018.txt"
s = requests.get(url).content
c = pd.read_csv(io.StringIO(s.decode('utf-8')), sep='|')

C:\Users\dargo\AppData\Local\Temp\ipykernel_692\1361649060.py:3: DtypeWarning: Columns (14,18,23,24,26,28,29,31,32,33,41) have mixed types. Specify dtype option on import or set low_memory=False.
  c = pd.read_csv(io.StringIO(s.decode('utf-8')), sep='|')


# Cleaning

In [ ]:
# Selection des colonnes utiles

dataset = c[['Date mutation',
         'Nature mutation',
         'Valeur fonciere',
         'Code voie',
         'Code postal',
         'Code departement',
         'Code commune',
         'Nombre de lots',
         'Code type local',
         'Surface reelle bati',
         'Nombre pieces principales',
         'Nature culture',
         'Nature culture speciale',
         'Surface terrain'
         ]]

In [ ]:
# suppression valeur fonciere nulle

dataset.dropna(subset=['Valeur fonciere'], inplace=True)

In [ ]:
# Passage de la colonnes date mutation en datetime puis en seconde pour avoir des int

dataset['Date mutation'] = pd.to_datetime(dataset['Date mutation'], format='mixed')
dataset['Date mutation'] = dataset['Date mutation'].apply(lambda x : x.timestamp())

In [ ]:
# Enumeration des différents types de mutation et labelisation en entier

mutation_values = dataset['Nature mutation'].drop_duplicates().values
for idx, i in enumerate(mutation_values) :
    dataset['Nature mutation'] = dataset['Nature mutation'].str.replace(i, str(idx))
    
dataset['Nature mutation']= dataset['Nature mutation'].apply(int)

# Conversion en str de toute la colonne avec remplissage des NaN, Labelisation + Ajout prise en compte code rivoli

dataset['Code voie'] = dataset['Code voie'].astype(str)
dataset['Code voie'].fillna('', inplace=True)
Code_Voie = preprocessing.LabelEncoder()
Code_Voie.fit(dataset['Code voie'])
dataset['Code voie'] =  Code_Voie.transform(dataset['Code voie'])

In [ ]:
# Remplissage des valeurs NaN puis labelisation

dataset['Type de voie'].fillna('', inplace=True)
Type_Voie = preprocessing.LabelEncoder()
Type_Voie.fit(dataset['Type de voie'])
dataset['Type de voie'] =  Type_Voie.transform(dataset['Type de voie'])

In [ ]:
# Remplissage des valeurs NaN

dataset['Surface reelle bati'].fillna(0, inplace=True)

In [ ]:
# Remplissage des valeurs NaN

dataset['Nombre pieces principales'].fillna(0, inplace=True)

In [ ]:
# Remplissage des valeurs NaN

dataset['Surface terrain'].fillna(0, inplace=True)

In [ ]:
dataset['Nature culture'].fillna('', inplace=True)
NC = preprocessing.LabelEncoder()
NC.fit(dataset['Nature culture'])
dataset['Nature culture'] =  NC.transform(dataset['Nature culture'])

In [ ]:
dataset['Nature culture speciale'].fillna('', inplace=True)
NCS = preprocessing.LabelEncoder()
NCS.fit(dataset['Nature culture speciale'])
dataset['Nature culture speciale'] =  NCS.transform(dataset['Nature culture speciale'])

In [ ]:
c.dtypes

# Y columns

In [ ]:
X = c.drop(['Valeur fonciere'], axis=1)
Y = c['Valeur fonciere'].str.replace(',', '.')
Y = Y.astype('float64')
Y = Y.astype(int)

In [ ]:
Y_10 = Y / 10
Y_10 = Y_10.astype(int)
Y_100 = Y_10 / 10
Y_100 = Y_100.astype(int)
Y_1000 = Y_100 / 10
Y_1000 = Y_1000.astype(int)
Y_10000 = Y_1000 / 10
Y_10000 = Y_10000.astype(int)

# Division

In [ ]:
X_test, X_train, Y_test, Y_train = train_test_split(X, Y, train_size=0.7, random_state=42)

In [ ]:
X_test_10, X_train_10, Y_test_10, Y_train_10 = train_test_split(X, Y_10, train_size=0.7, random_state=42)

In [ ]:
X_test_100, X_train_100, Y_test_100, Y_train_100 = train_test_split(X, Y_100, train_size=0.7, random_state=42)

In [ ]:
X_test_1000, X_train_1000, Y_test_1000, Y_train_1000 = train_test_split(X, Y_1000, train_size=0.7, random_state=42)

# Model training

## Random forest

In [ ]:
clf = RandomForestClassifier(n_estimators=100,max_depth=5, random_state=0)
clf.fit(X_train_1000,Y_train_1000)

In [ ]:
clf.predict_proba(X_test_1000)  